In [8]:
MAX_LEN = 512
BERT_MODEL = "bert-base-uncased"
# reduce lr on plateau

In [9]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from nltk.tokenize import TweetTokenizer


import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import re
import preprocessing

import random

In [10]:
class BERTModel(nn.Module):
    """
        BERT model with a dropout and linear layer with 2 outputs
    """
    def __init__(self, bert_model, num_classes=2, dropout_rate=0.3):
        super(BERTModel, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(bert_model)
        self.drop = nn.Dropout(dropout_rate)
        self.out = nn.Linear(self.bert.config.hidden_size, num_classes)


    def forward(self, ids, mask, token_type_ids):
        _, pooled_output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        output = self.drop(pooled_output)
        return self.out(output)

In [11]:
def train(data, model, optimizer, device):
    """
        Train the model for one epoch
    """
    model.train()
    running_loss = 0.0

    for batch_idx, d in enumerate(data):
        mask = d['mask'].to(device, dtype=torch.long)
        ids = d['ids'].to(device, dtype=torch.long)
        token_type_ids = d['token_type_ids'].to(device, dtype=torch.long)
        targets = d['targets'].to(device, dtype=torch.long)
        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)

        loss = torch.nn.CrossEntropyLoss(outputs, targets) # Calculate loss
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0 and batch_idx !=0:
            temp = f'Batch index = {batch_idx}\tRunning Loss = {running_loss/10}'
            print(temp)
            running_loss = 0.0

In [12]:
pos_path = 'data/twitter-datasets/train_pos.txt'
neg_path = 'data/twitter-datasets/train_neg.txt'

with open(pos_path, 'r') as f:
    pos_tweets = f.readlines()
with open(neg_path, 'r') as f:
    neg_tweets = f.readlines()

# TODO: preprocess data
# pos_tweets = preprocessing.remove_tags(pos_tweets)
# neg_tweets = preprocessing.remove_tags(neg_tweets)


pos_labels = [1 for _ in range(len(pos_tweets))]
neg_labels = [0 for _ in range(len(neg_tweets))]
labels = pos_labels + neg_labels
tweets = pos_tweets + neg_tweets

# Combine labels and tweets into a list of tuples
data = list(zip(tweets, labels))

# Shuffle the data
random.shuffle(data)

# Unpack the shuffled data back into separate lists
shuffled_tweets, shuffled_labels = zip(*data)

# Split the shuffled data into training and validation sets
train_tweets, val_tweets, train_labels, val_labels = train_test_split(shuffled_tweets, shuffled_labels, test_size=0.1)

In [13]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=True)
train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=MAX_LEN)
val_encodings = tokenizer(val_tweets, truncation=True, padding=True, max_length=MAX_LEN)


# tokenizer = TweetTokenizer()
# train_encodings = [tokenizer.tokenize(t) for t in train_tweets]
# val_encodings = [tokenizer.tokenize(t) for t in val_tweets]



class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)



In [14]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3  # Number of training epochs


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
criterion = CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(train_loader) * num_epochs)


# Function to calculate accuracy
def calc_accuracy(preds, labels):
    _, predictions = torch.max(preds, dim=1)
    correct = (predictions == labels).float()
    acc = correct.sum() / len(correct)
    return acc

all_labels = []
all_predictions = []

# Training Loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()
    total_train_loss = 0
    total_train_acc = 0

    for step, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = criterion(outputs.logits, batch['labels'])
        acc = calc_accuracy(outputs.logits, batch['labels'])
        print(f"Step {step+1}/{len(train_loader)} - Loss: {loss.item()}, Accuracy: {acc.item()}")
        total_train_loss += loss.item()
        total_train_acc += acc.item()

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_train_loss = total_train_loss / len(train_loader)
    avg_train_acc = total_train_acc / len(train_loader)
    print(f"Epoch {epoch+1} - Train loss: {avg_train_loss}, Accuracy: {avg_train_acc}")

    # Validation Loop
    model.eval()
    total_val_accuracy = 0
    total_val_loss = 0

    for batch in val_loader:
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = criterion(outputs.logits, batch['labels'])
            acc = calc_accuracy(outputs.logits, batch['labels'])

            total_val_loss += loss.item()
            total_val_accuracy += acc.item()

            # for overall metrics calculation
            preds = outputs.logits.argmax(dim=1).cpu().numpy()
            labels = batch['labels'].cpu().numpy()
            all_predictions.extend(preds)
            all_labels.extend(labels)

    avg_val_loss = total_val_loss / len(val_loader)
    avg_val_accuracy = total_val_accuracy / len(val_loader)
    print(f"Epoch {epoch+1} - Validation loss: {avg_val_loss}, Accuracy: {avg_val_accuracy}")

# After all epochs, calculate overall metrics
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')

print("Training complete!")
print(f"Overall Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Step 1/22500 - Loss: 0.7299100160598755, Accuracy: 0.375
Step 2/22500 - Loss: 0.7326957583427429, Accuracy: 0.5
Step 3/22500 - Loss: 0.8679541349411011, Accuracy: 0.125
Step 4/22500 - Loss: 0.7152673006057739, Accuracy: 0.5
Step 5/22500 - Loss: 0.6850508451461792, Accuracy: 0.5
Step 6/22500 - Loss: 0.6123498678207397, Accuracy: 0.75
Step 7/22500 - Loss: 0.8654451370239258, Accuracy: 0.25
Step 8/22500 - Loss: 0.7086740732192993, Accuracy: 0.5
Step 9/22500 - Loss: 0.6956126689910889, Accuracy: 0.5
Step 10/22500 - Loss: 0.6566214561462402, Accuracy: 0.625
Step 11/22500 - Loss: 0.6987295150756836, Accuracy: 0.375
Step 12/22500 - Loss: 0.6939436197280884, Accuracy: 0.5
Step 13/22500 - Loss: 0.6760130524635315, Accuracy: 0.5
Step 14/22500 - Loss: 0.667831540107727, Accuracy: 0.5
Step 15/22500 - Loss: 0.653003454208374, Accuracy: 0.5
Step 16/22500 - Loss: 0.64644855260849, Accuracy: 0.5
Step 17/22500 - Loss: 0.8006994724273682, Accuracy: 0.125
Step 18/22500 - Loss: 0.660507321357727

KeyboardInterrupt: 

### Possible improvement
1. Don't store all the scores in one variable, just store for each epoch and only keep the average score
2. save model for Each epoch

In [ ]:
torch.save(model, 'bert.pth')

In [ ]:
## predict
model = torch.load('bert.pth')

## use the test set
test_path = 'data/twitter-datasets/test_data.txt'
with open(test_path, 'r') as f:
    test_tweets = f.readlines()

test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=MAX_LEN)
test_dataset = TweetDataset(test_encodings, [0 for _ in range(len(test_tweets))])
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


model.eval()
predictions = []
for batch in test_loader:
    with torch.no_grad():
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = outputs.logits.argmax(dim=1).cpu().numpy()
        predictions.extend(preds)

In [ ]:
predictions = np.array(predictions)
predictions[predictions == 0] = -1

In [ ]:
print(predictions)

[-1 -1  1 ... -1  1 -1]


In [ ]:
import pandas as pd

submission = pd.DataFrame({'Id':range(1, len(predictions) + 1),'Prediction': predictions})
submission.to_csv('submission.csv', index=False)